Construindo um grafo automaticamente
====================================



## Introdução



Neste notebook nós vamos dar o primeiro passo para construir nossa rede neural artificial. Neste primeiro passo, nós vamos criar uma classe que gera automaticamente o nosso `grafo computacional`. O grafo computacional é o grafo que representa todas as operações matemáticas que ocorreram ao se computar um certo valor $y$. O grafo computacional é um passo necessário pois será baseado nele que iremos computar os gradientes locais necessários para realizar o `backpropagation`.



## Importações



In [25]:
from funcoes import plota_grafo

## Código e discussão



### Primeiros passos



A base de tudo será uma classe chamada `Valor`. Vamos começar pelo básico!



In [1]:
class Valor:
    def __init__(self, data):
        self.data = data

    def __repr__(self):
        return f"Valor(data={self.data})"

Vamos testar nossa classe!



In [2]:
x1 = Valor(60)
print(x1)
print(x1.data)

Valor(data=60)
60


### Os dunders `__add__` e `__mul__`



Observe que não conseguimos adicionar nem multiplicar objetos criados com a classe `Valor`.



In [3]:
a = Valor(10)
b = Valor(5)

In [4]:
print(a + b)

TypeError: unsupported operand type(s) for +: 'Valor' and 'Valor'

In [5]:
print(a * b)

TypeError: unsupported operand type(s) for *: 'Valor' and 'Valor'

Ué&#x2026; porque não conseguimos? Não conseguimos pois o Python (ainda) não é vidente. Ele lá vai saber como adicionar ou multiplicar algo que você criou? Pra você parece óbvio que valores podem ser adicionados ou multiplicados, mas para o Python ele nem sabe o que significa a palavra `Valor`&#x2026;

Como sempre, temos que contar para o programa o que queremos que aconteça quando usarmos os operadores `+` e `*`. Quem faz isso são os dunders `__add__` e `__mul__`.



In [7]:
class Valor:
    def __init__(self, data):
        self.data = data

    def __repr__(self):
        return f"Valor(data={self.data})"

    def __add__(self, outro_valor):
        saida = Valor(self.data + outro_valor.data)
        return saida

    def __mul__(self, outro_valor):
        saida = Valor(self.data * outro_valor.data)
        return saida

Vamos testar!



In [8]:
a = Valor(10)
b = Valor(5)

print(a+b)
print(a*b)

Valor(data=15)
Valor(data=50)


### Registrando os progenitores



Nosso objetivo é construir um grafo computacional. Em um grafo computacional, um certo vértice pode ter um ou mais vértices progenitores (são seus pais/mães). Nós não podemos perder essa informação quando formos construir um grafo, então precisamos incluir essa informação na nossa classe.



In [9]:
class Valor:
    def __init__(self, data, progenitor=()):
        self.data = data
        self.progenitor = progenitor

    def __repr__(self):
        return f"Valor(data={self.data})"

    def __add__(self, outro_valor):
        data = self.data + outro_valor.data
        progenitor = (self, outro_valor)
        saida = Valor(data, progenitor)
        return saida

    def __mul__(self, outro_valor):
        data = self.data * outro_valor.data
        progenitor = (self, outro_valor)
        saida = Valor(data, progenitor)
        return saida

Vamos testar!



In [13]:
a = Valor(10)
b = Valor(5)

print(a.data)
print(a.progenitor) # n tem

c = a + b

#d = a * b

print (c)
print(c.progenitor) # agora tem, porque fizemos uma ação em cima


10
()
Valor(data=15)
(Valor(data=10), Valor(data=5))


### Registrando o operador mãe



Em um grafo computacional, um vértice pode ter um operador mãe. O operador mãe é o operador que foi usado para gerar o vértice.



In [26]:
class Valor:
    def __init__(self, data, progenitor=(), operador_mae=""):
        self.data = data
        self.progenitor = progenitor
        self.operador_mae = operador_mae

    def __repr__(self):
        return f"Valor(data={self.data})"

    def __add__(self, outro_valor):
        data = self.data + outro_valor.data
        progenitor = (self, outro_valor)
        operador_mae = "+"
        saida = Valor(data, progenitor, operador_mae)
        return saida

    def __mul__(self, outro_valor):
        data = self.data * outro_valor.data
        progenitor = (self, outro_valor)
        operador_mae = "*"
        saida = Valor(data, progenitor, operador_mae)
        return saida

Vamos testar!



In [27]:
a = Valor(10)
b = Valor(5)

print(a.data)
print(a.progenitor)

c = a + b

d = a * b

print (c)
print(c.progenitor)
print (c.operador_mae) 

10
()
Valor(data=15)
(Valor(data=10), Valor(data=5))
+


### Plotando o primeiro grafo



Vamos plotar nosso primeiro grafo!



In [28]:
a = Valor(2)
b = Valor(-3)
c = Valor(10)

d = a * b
e = d + c

grafo = plota_grafo(e)
print(grafo)

digraph {
	graph [rankdir=LR]
	2089065069104 [label="{ data 4.0000 }" shape=record]
	"2089065069104+" [label="+"]
	"2089065069104+" -> 2089065069104
	2089065421600 [label="{ data 2.0000 }" shape=record]
	2089065420640 [label="{ data -3.0000 }" shape=record]
	2089065422224 [label="{ data 10.0000 }" shape=record]
	2089065422272 [label="{ data -6.0000 }" shape=record]
	"2089065422272*" [label="*"]
	"2089065422272*" -> 2089065422272
	2089065421600 -> "2089065422272*"
	2089065422224 -> "2089065069104+"
	2089065422272 -> "2089065069104+"
	2089065420640 -> "2089065422272*"
}



### Registrando o rótulo



Nosso grafo seria mais legível se tivéssemos rótulos indicando o que é cada vértice. Vamos incluir essa informação na nossa classe.



In [29]:
class Valor:
    def __init__(self, data, progenitor=(), operador_mae="", rotulo=""):
        self.data = data
        self.progenitor = progenitor
        self.operador_mae = operador_mae
        self.rotulo = rotulo

    def __repr__(self):
        return f"Valor(data={self.data})"

    def __add__(self, outro_valor):
        data = self.data + outro_valor.data
        progenitor = (self, outro_valor)
        operador_mae = "+"
        saida = Valor(data, progenitor, operador_mae)
        return saida

    def __mul__(self, outro_valor):
        data = self.data * outro_valor.data
        progenitor = (self, outro_valor)
        operador_mae = "*"
        saida = Valor(data, progenitor, operador_mae)
        return saida

Vamos testar!



In [30]:
a = Valor(2, rotulo="a")
b = Valor(-3, rotulo="b")
c = Valor(10, rotulo="c")

d = a * b
e = d + c

# o python nao sabia qual é o nome, mudamos na mão (faz sentido)
d.rotulo = "d"
e.rotulo = "e"

grafo = plota_grafo(e)
print(grafo)

digraph {
	graph [rankdir=LR]
	2089065443040 [label="{ e | data 4.0000 }" shape=record]
	"2089065443040+" [label="+"]
	"2089065443040+" -> 2089065443040
	2089065440016 [label="{ a | data 2.0000 }" shape=record]
	2089065440112 [label="{ b | data -3.0000 }" shape=record]
	2089065440208 [label="{ d | data -6.0000 }" shape=record]
	"2089065440208*" [label="*"]
	"2089065440208*" -> 2089065440208
	2089065442272 [label="{ c | data 10.0000 }" shape=record]
	2089065440208 -> "2089065443040+"
	2089065440016 -> "2089065440208*"
	2089065442272 -> "2089065443040+"
	2089065440112 -> "2089065440208*"
}



### Refazendo o grafo que fizemos na aula anterior



Na aula anterior nós fizemos um grafo computacional para aprender como funciona o backpropagation. Vamos refazer ele aqui!



In [33]:
x1 = Valor (60, rotulo = "x1")
x2 = Valor (24, rotulo = "x2")
w1 = Valor (10, rotulo = "w1")
w2 = Valor (5, rotulo = "w2")
w3 = Valor (2, rotulo = "w3")
b = Valor (7, rotulo = "b")

s1 = x1*w1
s1.rotulo = 's1'

s2 = x2 * w2
s2.rotulo = 's2'

n = s1 + s2 
n.rotulo = 'n'

k = n+b
k. rotulo = 'k'

y = k * w3
y.rotulo = 'y'

grafo = plota_grafo(y)
print(grafo)

digraph {
	graph [rankdir=LR]
	2089065888784 [label="{ x2 | data 24.0000 }" shape=record]
	2089065888304 [label="{ b | data 7.0000 }" shape=record]
	2089065422464 [label="{ k | data 727.0000 }" shape=record]
	"2089065422464+" [label="+"]
	"2089065422464+" -> 2089065422464
	2089065888400 [label="{ w3 | data 2.0000 }" shape=record]
	2089065888496 [label="{ w2 | data 5.0000 }" shape=record]
	2089065888016 [label="{ s2 | data 120.0000 }" shape=record]
	"2089065888016*" [label="*"]
	"2089065888016*" -> 2089065888016
	2089065420064 [label="{ y | data 1454.0000 }" shape=record]
	"2089065420064*" [label="*"]
	"2089065420064*" -> 2089065420064
	2089065888064 [label="{ n | data 720.0000 }" shape=record]
	"2089065888064+" [label="+"]
	"2089065888064+" -> 2089065888064
	2089065888592 [label="{ w1 | data 10.0000 }" shape=record]
	2089065888688 [label="{ x1 | data 60.0000 }" shape=record]
	2089065888208 [label="{ s1 | data 600.0000 }" shape=record]
	"2089065888208*" [label="*"]
	"2089065888208*" -> 

## Conclusão

Os grafos são um modo para que visualizarmos as alterações nas nossas classes, o que está acontecendo toda vez que aplicamos ou modificamos o objeto que estamos analisando. Com eles podemos realizar operações que matematicamente não poderiam ser feitas, mas como podemos definir o que cada caracter dunder faz, podemos realizar operações do modo que melhor se adequa ao problema. Nesse caso, conseguimos somar ou multiplicar valores que estão dentra da nossa classe. Para facilitar, damos rótumos para valores, assim fica ainda mais simples atuar métodos ou oporadores neles. Assim, fica mais claro como um valor, depois de passar por uma 'transformação' que é o operador mãe, passando por uma série de novos vértices. 


## Playground

